## Recommender Systems Collaborative Filtering 

In [1]:
import numpy as np
from numpy import loadtxt
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

%matplotlib inline

2024-07-16 10:11:05.356103: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-16 10:11:07.747478: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
class rs:
    dataset_path = '../../datasets/recommendation_systems/'

    @staticmethod
    def load_data():
        file = open(rs.dataset_path + 'small_movies_X.csv', 'rb')
        X = loadtxt(file, delimiter=",")

        file = open(rs.dataset_path + 'small_movies_W.csv', 'rb')
        W = loadtxt(file, delimiter=",")

        file = open(rs.dataset_path + 'small_movies_b.csv', 'rb')
        b = loadtxt(file, delimiter=",")

        b = b.reshape(1, -1)

        num_movies, num_features = X.shape
        num_users, _ = W.shape
        return (X, W, b, num_movies, num_features, num_users)

    @staticmethod
    def load_ratings():
        file = open(rs.dataset_path + 'small_movies_Y.csv', 'rb')
        Y = loadtxt(file, delimiter=",")

        file = open(rs.dataset_path + 'small_movies_R.csv', 'rb')
        R = loadtxt(file, delimiter=",")

        return Y, R
    
    
    def cofi_cost_func(W, X, b, Y, R, lambda_):
        nm, nu = Y.shape
        J = 0
        for j in range(nu):
            w = W[j, :]
            b_j = b[0, j]

            for i in range(nm):
                x = X[i, :]
                y = Y[i, j]
                r = R[i, j]
                J += np.square(r * ((np.dot(w, x) + b) - y))

        J /= 2
        J += (lambda_/2) * (np.sum(np.square(W)) + np.sum(np.square(X)))

        return J
    
    def cofi_cost_func_v(X, W, b, Y, R, lambda_):
        """
        Returns the cost for the content-based filtering
        Vectorized for speed. Uses tensorflow operations to be compatible with custom training loop.
        Args:
        X (ndarray (num_movies,num_features)): matrix of item features
        W (ndarray (num_users,num_features)) : matrix of user parameters
        b (ndarray (1, num_users)            : vector of user parameters
        Y (ndarray (num_movies,num_users)    : matrix of user ratings of movies
        R (ndarray (num_movies,num_users)    : matrix, where R(i, j) = 1 if the i-th movies was rated by the j-th user
        lambda_ (float): regularization parameter
        Returns:
        J (float) : Cost
        """
        j = (tf.linalg.matmul(X, tf.transpose(W)) + b - Y)*R
        J = 0.5 * tf.reduce_sum(j**2) + (lambda_/2) * (tf.reduce_sum(X**2) + tf.reduce_sum(W**2))
        return J


In [3]:
X, W, b, num_movies, num_features, num_users = rs.load_data()
Y, R = rs.load_ratings()

In [4]:
num_users_r = 4
num_movies_r = 5
num_features_r = 3


X_r = X[:num_movies_r, :num_features_r]
W_r = W[:num_users_r, :num_features_r]
b_r = b[0, :num_users_r].reshape(1, -1)
Y_r = Y[:num_movies_r, :num_users_r]
R_r = R[:num_movies_r, :num_users_r]

# Evaluate cost function
J = rs.cofi_cost_func_v(X_r, W_r, b_r, Y_r, R_r, 0)
print(f"Cost: {J:0.2f}")

Cost: 13.67


2024-07-16 10:11:12.700983: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-16 10:11:13.178279: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
